In [ ]:
from typing import Dict, Any, List
from datasets import load_dataset, concatenate_datasets
import evaluate
import json
import re
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# Prepare Data

In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
tokenizer.add_tokens(["<"])

1

## Spider

In [3]:
# prepare table for cosql and sparc
with open("cosql_dataset/tables.json") as json_file:
    json_data = json.load(json_file)
    
database = dict()

for db in json_data:
    database[db["db_id"]] = dict()
    for table in db["table_names_original"]:
        database[db["db_id"]][table] = []
    for column in db["column_names_original"]:
        table_id = column[0]
        col_name = column[1]
        if table_id != -1:
            database[db["db_id"]][db["table_names_original"][table_id]].append(col_name)

In [ ]:
spider = load_dataset("spider")

In [5]:
spider

DatasetDict({
    train: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 1034
    })
})

In [6]:
def db_id_to_table(db_id):
    global database
    db = database[db_id]
    table = [f"{table}({','.join(db[table])})" for table in db]
    table = ", ".join(table)
    return table

In [7]:
def clean_text(text):
    text = text.replace("\xa0", " ").strip()
    text = re.sub(r'\s+', ' ', text)
    return text

def clean_query(query):
    query = clean_text(query).replace(" , ", ", ")
    if query[-1] == ";":
        query = query[:-1]
    return query

def get_prompt(tables, question):
    question = clean_text(question)
    prompt = f"""convert question and table into SQL query. tables: {tables}. question: {question}"""
    return prompt

In [8]:
def spider_preprocess_function(examples):
    tables = db_id_to_table(examples["db_id"])
    examples["query"] = clean_query(examples["query"])
    examples["prompt"] = get_prompt(tables, examples["question"])
    return examples

def spider_tokenize(examples):
    return tokenizer(examples["prompt"], text_target=examples["query"], max_length=tokenizer.model_max_length, truncation=True)

In [ ]:
spider_train = spider["train"].map(spider_preprocess_function, remove_columns=['db_id', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'])
spider_val = spider["validation"].map(spider_preprocess_function, remove_columns=['db_id', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'])

In [10]:
spider_val[43]

{'query': 'select count(*) from concert where stadium_id = (select stadium_id from stadium order by capacity desc limit 1)',
 'prompt': 'convert question and table into SQL query. tables: stadium(Stadium_ID,Location,Name,Capacity,Highest,Lowest,Average), singer(Singer_ID,Name,Country,Song_Name,Song_release_year,Age,Is_male), concert(concert_ID,concert_Name,Theme,Stadium_ID,Year), singer_in_concert(concert_ID,Singer_ID). question: Find the number of concerts happened in the stadium with the highest capacity .'}

In [ ]:
spider_train = spider_train.map(spider_tokenize, remove_columns=['prompt', 'query'])
spider_val = spider_val.map(spider_tokenize, remove_columns=['prompt', 'query'])

# CoSQL & SPARC

In [12]:
# prepare table for cosql and sparc
# with open("/mnt/c/Users/Jui/Projects/data/cosql_dataset/tables.json") as json_file:
#     json_data = json.load(json_file)
    
# database = dict()

# for db in json_data:
#     database[db["db_id"]] = dict()
#     for table in db["table_names_original"]:
#         database[db["db_id"]][table] = []
#     for column in db["column_names_original"]:
#         table_id = column[0]
#         col_name = column[1]
#         if table_id != -1:
#             database[db["db_id"]][db["table_names_original"][table_id]].append(col_name)

In [13]:
# prepare data
def json_path_to_dataframe(path):
    with open(path) as json_file:
        json_data = json.load(json_file)
        
    df = pd.DataFrame(columns=["prompt", "query"])

    for row in json_data:
        question = row["final"]["utterance"]
        query = row["final"]["query"]

        tables = db_id_to_table(row["database_id"])
        query = clean_query(query)
        prompt = get_prompt(tables, question)

        df = pd.concat([df, pd.DataFrame({
            "prompt": [prompt],
            "query": [query]
        })], axis=0, ignore_index=True)
    return df

# cosql
json_path_to_dataframe("/mnt/c/Users/Jui/Projects/data/cosql_dataset/sql_state_tracking/cosql_dev.json").to_csv("cosql_dev.csv", index=False)
json_path_to_dataframe("/mnt/c/Users/Jui/Projects/data/cosql_dataset/sql_state_tracking/cosql_train.json").to_csv("cosql_train.csv", index=False)

# sparc
json_path_to_dataframe("/mnt/c/Users/Jui/Projects/data/sparc/dev.json").to_csv("sparc_dev.csv", index=False)
json_path_to_dataframe("/mnt/c/Users/Jui/Projects/data/sparc/train.json").to_csv("sparc_train.csv", index=False)

In [ ]:
cosql = load_dataset("csv", data_files={"dev": "cosql_dev.csv", "train": "cosql_train.csv"})
sparc = load_dataset("csv", data_files={"dev": "sparc_dev.csv", "train": "sparc_train.csv"})

In [15]:
cosql

DatasetDict({
    dev: Dataset({
        features: ['prompt', 'query'],
        num_rows: 293
    })
    train: Dataset({
        features: ['prompt', 'query'],
        num_rows: 2159
    })
})

In [16]:
cosql["dev"][32]

{'prompt': 'convert question and table into SQL query. tables: players(player_id,first_name,last_name,hand,birth_date,country_code), matches(best_of,draw_size,loser_age,loser_entry,loser_hand,loser_ht,loser_id,loser_ioc,loser_name,loser_rank,loser_rank_points,loser_seed,match_num,minutes,round,score,surface,tourney_date,tourney_id,tourney_level,tourney_name,winner_age,winner_entry,winner_hand,winner_ht,winner_id,winner_ioc,winner_name,winner_rank,winner_rank_points,winner_seed,year), rankings(ranking_date,ranking,player_id,ranking_points,tours). question: Find the average age of losers and winners of all matches.',
 'query': 'SELECT avg(loser_age), avg(winner_age) FROM matches'}

In [17]:
sparc

DatasetDict({
    dev: Dataset({
        features: ['prompt', 'query'],
        num_rows: 422
    })
    train: Dataset({
        features: ['prompt', 'query'],
        num_rows: 3034
    })
})

In [18]:
sparc["dev"][0]

{'prompt': 'convert question and table into SQL query. tables: airlines(uid,Airline,Abbreviation,Country), airports(City,AirportCode,AirportName,Country,CountryAbbrev), flights(Airline,FlightNo,SourceAirport,DestAirport). question: What country is Jetblue Airways affiliated with?',
 'query': 'SELECT Country FROM AIRLINES WHERE Airline = "JetBlue Airways"'}

In [19]:
def general_tokenize(examples):
    return tokenizer(examples["prompt"], text_target=examples["query"], max_length=tokenizer.model_max_length, truncation=True)

In [20]:
cosql_train = cosql["train"].map(general_tokenize, remove_columns=['prompt', 'query'])
cosql_val = cosql["dev"].map(general_tokenize, remove_columns=['prompt', 'query'])

sparc_train = sparc["train"].map(general_tokenize, remove_columns=['prompt', 'query'])
sparc_val = sparc["dev"].map(general_tokenize, remove_columns=['prompt', 'query'])

Map:   0%|          | 0/2159 [00:00<?, ? examples/s]

Map:   0%|          | 0/293 [00:00<?, ? examples/s]

Map:   0%|          | 0/3034 [00:00<?, ? examples/s]

Map:   0%|          | 0/422 [00:00<?, ? examples/s]

## Mixed

In [21]:
from datasets import concatenate_datasets

In [22]:
train_dataset = concatenate_datasets([spider_train, cosql_train, sparc_train])
val_dataset = concatenate_datasets([spider_val, cosql_val, sparc_val])

In [23]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 12193
})

In [24]:
val_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1749
})

## Training

In [25]:
len(tokenizer)

32101

In [26]:
# declare model
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model.resize_token_embeddings(len(tokenizer))
model = model.cuda()

In [27]:
model.get_memory_footprint()

990145536

In [28]:
# declare data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [29]:
# declare training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    evaluation_strategy="steps",
    eval_steps=381,
    logging_steps=381,
    save_strategy="steps",
    save_steps=381,
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    # gradient_checkpointing=True,
    # warmup_ratio=0.01,
    # weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,
    # fp16=True,
    # bf16=True, 
    # predict_with_generate=True,
    # generation_max_length=512,
    # generation_num_beams=None,
    # lr_scheduler_type="cosine",
    # dataloader_num_workers=2,
    greater_is_better=False,
    # metric_for_best_model="eval_loss",
)

In [30]:
# import numpy as np


# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     # Decode generated summaries into text
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     # Decode reference summaries into text
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     # ROUGE expects a newline after each sentence
#     decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
#     decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
#     # Compute ROUGE scores
#     result = rouge_score.compute(
#         predictions=decoded_preds, references=decoded_labels, use_stemmer=True
#     )
#     # Extract the median scores
#     result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
#     return {k: round(v, 4) for k, v in result.items()}

In [31]:
# declare trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
) # you can evaluate by using compute_metrics function above, but I comment out for the faster training loop

In [ ]:
trainer.train()
# trainer.train(resume_from_checkpoint="./results/checkpoint-13000")

## Upload Model

In [50]:
from huggingface_hub import notebook_login

notebook_login()

In [51]:
model.push_to_hub(repo_id="juierror/flan-t5-text2sql-with-schema-v2")

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/juierror/flan-t5-text2sql-with-schema-v2/commit/01a1b7a8ee9722b648c2add3f2477a1824b666a2', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='01a1b7a8ee9722b648c2add3f2477a1824b666a2', pr_url=None, pr_revision=None, pr_num=None)

In [52]:
tokenizer.push_to_hub(repo_id="juierror/flan-t5-text2sql-with-schema-v2")

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/juierror/flan-t5-text2sql-with-schema-v2/commit/0d80cff0d64cd6833cce6ac61ede8af43340b339', commit_message='Upload tokenizer', commit_description='', oid='0d80cff0d64cd6833cce6ac61ede8af43340b339', pr_url=None, pr_revision=None, pr_num=None)